# Es 1.. 2.1

In [76]:
import networkx as nx
import pandas as pd
import json
import ijson
import time
import csv
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import ast
from itertools import combinations
from collections import deque

In [77]:
citation_graph = nx.DiGraph()
collaboration_graph = nx.Graph()


In [78]:
df_sorted =pd.read_csv('C:/Users/Gabriele/Desktop/università/Magistrale/DATA SCIENCE/ADM/HW5/dataset_sorted_top10000.tsv',sep='\t')

In [79]:
df_sorted.tail()

,id,title,year,authors,n_citation,doc_type,reference_count,references,doi,publisher
9995,1964830323,An overview of JML tools and applications,2005.0,"[{'name': 'Lilian Burdy', 'org': 'INRIA Sophia...",596.0,Conference,82.0,1486696980;1489778371;1492315860;1498946538;14...,https://doi.org/10.1007/s10009-004-0167-4,Springer-Verlag
9996,2101699859,Countering code-injection attacks with instruc...,2003.0,"[{'name': 'Gaurav S. Kc', 'org': 'Columbia Uni...",596.0,Conference,42.0,186343359;1481758559;1499992849;1508969946;151...,https://doi.org/10.1145/948109.948146,ACM
9997,2124609748,Gaussian Process Dynamical Models for Human Mo...,2008.0,"[{'name': 'J.M. Wang', 'org': 'University of T...",596.0,Conference,61.0,1505866674;1546296670;1598702468;1643263348;19...,https://doi.org/10.1109/TPAMI.2007.1167,IEEE Computer Society
9998,2147343704,EVENODD: an efficient scheme for tolerating do...,1995.0,"[{'name': 'M. Blaum', 'org': 'IBM Almaden Rese...",596.0,Journal,9.0,1530042190;1531975040;1820898047;1829547464;20...,https://doi.org/10.1109/12.364531,IEEE Computer Society
9999,2093212899,Simultaneous structure and texture image inpai...,2003.0,"[{'name': 'M. Bertalmio', 'org': 'Dept. de Tec...",596.0,Journal,22.0,1488881187;1565233179;1569587969;1983661653;19...,https://doi.org/10.1109/TIP.2003.815261,IEEE


In [80]:
def fill_nodes_citation(data, G):
    for _, row in data.iterrows():
        paper_id = row['id']
        references_str = row['references']

        # Convert the references string to a list of integers
        references = [int(ref) for ref in str(references_str).split(';') if ref.isdigit()]

        # Add edges to the graph if the reference exists in the dataset
        for ref in references:
            if ref in data['id'].values:
                G.add_edge(paper_id, ref)


In [81]:
fill_nodes_citation(df_sorted, citation_graph)

In [82]:
def fill_nodes_collaboration(data, G):
    for _, row in data.iterrows():
        authors_str = row['authors']
        title = row['id']  

        try:
            authors = ast.literal_eval(authors_str)
            # Extract a unique identifier for each author (e.g., name or ID)
            author_ids = [author.get('name') or author.get('id') for author in authors]
        except ValueError:
            author_ids = []

        author_pairs = list(combinations(author_ids, 2))

        for author_pair in author_pairs:
            if G.has_edge(*author_pair):
                G[author_pair[0]][author_pair[1]]['weight'] = G[author_pair[0]][author_pair[1]]['weight'] + 1 if 'weight' in G[author_pair[0]][author_pair[1]] else 1
            else:
                G.add_edge(author_pair[0], author_pair[1], weight=1, titles=title)


In [83]:
fill_nodes_collaboration(df_sorted, collaboration_graph)

# 2.1

### Functionality 1 - Graph's features
This function should examine a graph and report on some of its features. The input and report that this function should produce are shown below. 

Input: 
- The graph
- The name of the graph

Output: 
- The number of the nodes in the graph
- The number of the edges in the graph
- The graph density
- The graph degree distribution
- The average degree of the graph
- The graph hubs (hubs are nodes having degrees more extensive than the 95th percentile of the degree distribution)
- Whether the graph is dense or sparse

-----------------------------------------------------------

In [84]:
def functionality_1(graph, graph_name):
    if graph_name != "citation_graph" and graph_name != "collaboration_graph":
        raise ValueError("Choose 'citation_graph' or 'collaboration_graph' as the second input value")
    
    # Number of nodes
    num_nodes = graph.number_of_nodes()

    # Number of edges
    num_edges = graph.number_of_edges()

    # Graph density
    density = nx.density(graph)

    # Degree distribution
    degree_sequence = [d for n, d in graph.degree()]
    degree_distribution = dict(zip(*np.unique(degree_sequence, return_counts=True)))

    # Average degree
    avg_degree = np.mean(degree_sequence)

    # 95th percentile of the degree distribution
    percentile_95 = np.percentile(degree_sequence, 95)

    # Graph hubs (nodes with degrees > 95th percentile)
    hubs = [node for node, degree in graph.degree() if degree > percentile_95]

    # Whether the graph is dense or sparse
    graph_type = "dense" if density >= 0.5 else "sparse"
    
    return num_nodes, num_edges, density, degree_distribution, avg_degree, hubs, graph_type


### Functionality 2 - Nodes' contribution 
Using this functionality, you will identify the papers/authors who have significantly contributed to this field of study. For this analysis, focusing solely on the number of citations for the paper or the number of collaborations of the authors can be misleading. You will examine this using various centrality measurements. 

Input:
- The graph
- A node of the graph (paper/author)
- The name of the graph

Output: 
- The centrality of the node, calculated based on the following centrality measurements:
   - Betweeness
   - PageRank
   - ClosenessCentrality
   - DegreeCentrality

------------------------------------------------------------------------------

In [85]:
def functionality_2(graph, node, graph_name):
    
    if graph_name != "citation_graph" and graph_name != "collaboration_graph":
        raise ValueError("Choose 'citation_graph' or 'collaboration_graph' as the second input value")
    
    if node not in graph.nodes:
        raise ValueError(f"node '{node}' not found in the graph.")
    
    
    centrality_measures = {}

    # Betweenness Centrality
    betweenness_centrality = nx.betweenness_centrality(graph)
    centrality_measures['Betweenness'] = betweenness_centrality.get(node, 0)

    # PageRank
    pagerank_centrality = nx.pagerank(graph)
    centrality_measures['PageRank'] = pagerank_centrality.get(node, 0)

    # Closeness Centrality
    closeness_centrality = nx.closeness_centrality(graph)
    centrality_measures['ClosenessCentrality'] = closeness_centrality.get(node, 0)

    # Degree Centrality
    degree_centrality = nx.degree_centrality(graph)
    centrality_measures['DegreeCentrality'] = degree_centrality.get(node, 0)

    """"""""""
    print(f"Centrality Measures for {node} in {graph_name}:")
    for measure, value in centrality_measures.items():
        print(f"{measure}: {value}")
    """""""""""
    
    return centrality_measures


### Functionality 3 - Shortest ordered walk  

Input:  
- The graph data 
- A sequence of authors\_a = [a\_2, ..., a\_{n-1}]
- Initial node a\_1 and an end node a\_n
- $N$: denoting the top $N$ authors whose data should be considered
 
Output: 
- The shortest walk of collaborations you need to read to get from author a\_1 to author a\_n and the papers you need to cross to realize this walk.

---------------------------------------------------------------------

This helper function is useful in functionality 3 and 4 in order to get the top N authors based on papers published

In [86]:
for author in collaboration_graph.nodes:
    collaboration_graph.nodes[author]['papers_published'] = len(list(collaboration_graph.neighbors(author)))
def get_top_nodes(graph, N):
        return sorted(graph.nodes, key=lambda x: graph.nodes[x]['papers_published'], reverse=True)[:N]

The function uses a Breadth-First Search (BFS) approach to explore the graph and find the shortest path between two nodes.
It then performs BFS to find the shortest path and associated titles from the starting node to the first intermediate node.
Subsequently, it iterates through the intermediate nodes, calculating the shortest path and titles for each pair.
Finally, it computes the shortest path and titles from the last intermediate node to the destination node, appending the results to the combined_path dictionary.
If any segment of the path is not found, the function returns a corresponding error message.
The final result is a dictionary containing the combined path and associated titles.


In [87]:
def functionality_3(graph, start, end, intermediate_nodes, N):
    combined_path = {'path': [], 'titles': []}

    # Helper function to perform BFS
    def BFS(graph, start, end, top_nodes):
        path_queue = deque()

        # Each element in the queue is a tuple (current_path, titles_list)
        temp_path = ([start], [])
        path_queue.append(temp_path)

        while path_queue:
            # Dequeue the next path and its associated titles
            current_path, current_titles = path_queue.popleft()
            last_node = current_path[-1]

            # Check if the last node in the path is the target
            if last_node == end:
                return current_path, current_titles

            # Explore neighbors of the last node
            for link_node, edge_data in graph[last_node].items():
                # Check if the neighbor is not already in the current path and is in the top N nodes
                if link_node not in current_path and link_node in top_nodes:
                    # Extend the path and titles with the new neighbor and its associated titles
                    new_path = current_path + [link_node]
                    new_titles = current_titles + [edge_data.get('titles', [])]
                    path_queue.append((new_path, new_titles))

        # If no valid path is found
        return [], []


    # Get the top N nodes based on papers published
    top_nodes = get_top_nodes(graph, N)

    # Calculate the first path and titles
    first_path, first_titles = BFS(graph, start, intermediate_nodes[0], top_nodes)
    combined_path['path'] += first_path
    combined_path['titles'] += first_titles

    if not first_path:
        return "There is no such path."

    start = intermediate_nodes[0]

    # Iterate through intermediate nodes and calculate the shortest path for each pair
    for intermediate_node in intermediate_nodes[1:]:
        path, titles = BFS(graph, start, intermediate_node, top_nodes)
        combined_path['path'] += path[1:]
        combined_path['titles'] += titles

        if not path:
            return "There is no such path."

        start = intermediate_node

    # Calculate the shortest path from the last intermediate node to the end
    last_path, last_titles = BFS(graph, intermediate_nodes[-1], end, top_nodes)
    combined_path['path'] += last_path[1:]
    combined_path['titles'] += last_titles

    if not last_path:
        return "There is no such path."

    return combined_path


### Functionality 4 - Disconnecting Graphs

Input: 
- The graph data 
- authorA: a paper to which will relate sub-graph G\_a
- authorB: a paper to which will relate sub-graph G\_b
- $N$: denoting the top $N$ authors that their data should be considered

Output:
- The minimum number of edges (by considering their weights) required to disconnect the original graph in two disconnected subgraphs: G\_a and G\_b.

---------------------------------------------------------------------------------------

The Ford-Fulkerson algorithm is used to find the maximum flow in a flow network. To disconnect graphs, one can modify this algorithm by identifying and removing edges with the maximum flow. This process is repeated until the remaining graph becomes disconnected

Directed graphs provide a clear flow direction for the Ford-Fulkerson algorithm, essential for determining and augmenting flow paths. Undirected graphs lack this directional information, hindering the algorithm's ability to establish consistent flow directions and compute maximum flows effectively.

This function converts an undirected graph into a directed graph.
It ensures consistent flow directions for the Ford-Fulkerson algorithm.


In [88]:

def convert_to_directed(original_graph):
    # Create a new directed graph
    directed_graph = nx.DiGraph()
    
    # Iterate over the edges of the original graph
    for source, target, edge_data in original_graph.edges(data=True):
        edge_weight = edge_data.get('weight')
        # Create directed edges in both directions
        directed_graph.add_edge(source, target, weight=edge_weight)
        directed_graph.add_edge(target, source, weight=edge_weight)  
    
    return directed_graph



This function performs a Breadth-First Search (BFS) to find augmenting paths.
It updates the parents dictionary to keep track of the path from start to end.


In [89]:

def BFS(graph, parents, start, end):
    visited = set()
    queue = deque([start])
    visited.add(start)

    while queue:
        current_node = queue.popleft()

        # Check if the current node is the end
        if current_node == end:
            break

        # Explore neighbors for potential paths
        for neighbor in graph.neighbors(current_node):
            # Check if the edge has remaining capacity and has not been visited yet
            if neighbor not in visited and graph[current_node][neighbor].get('weight') > 0:
                queue.append(neighbor)
                visited.add(neighbor)
                parents[neighbor] = current_node


This function implements the modified version of the Ford-Fulkerson algorithm to find the maximum flow in a graph in order to find the minimum cut.
It modifies the graph by iteratively identifying augmenting paths and updating residual capacities.


In [90]:

def ford_fulkerson(graph, start, end):
    # Create the residual graph, ensuring it is directed
    residual_graph = convert_to_directed(graph.copy())
    
    # Initialization
    parents = {}
    max_flow = 0

    # Continue until no more augmenting paths can be found
    while True:
        parents = {}
        BFS(residual_graph, parents, start, end)
        
        if end not in parents:
            break
        
        path_flow = float('inf')
        current_node = end
        
        # Update the minimum residual capacity along the path and accumulate the maximum flow
        for current_node in iter(lambda: parents[current_node], start):
            path_flow = min(path_flow, residual_graph[parents[current_node]][current_node]['weight'])

        max_flow += path_flow

        # Update residual capacities along the path
        current_node = end
        while current_node != start:
            parent_node = parents[current_node]
            residual_graph[parent_node][current_node]['weight'] -= path_flow
            residual_graph[current_node][parent_node]['weight'] += path_flow
            current_node = parent_node
        

    # Identify edges with zero residual capacity in the forward direction, forming the minimum cut
    cut_edges = set()
    for u in residual_graph.nodes():
        for v, edge_data in residual_graph[u].items():
            if edge_data['weight'] == 0:
                cut_edges.add((u, v))

    return cut_edges


In [91]:
def functionality_4(graph, authorA, authorB, N):
    # Get the top N nodes based on papers published
    top_nodes = get_top_nodes(graph, N)
    subgraph = graph.subgraph(set(top_nodes))

    # Check if authorA and authorB are in the graph
    if authorA not in subgraph.nodes:
        raise ValueError(f"Author '{authorA}' not found in the graph.")
    elif authorB not in subgraph.nodes:
        raise ValueError(f"Author '{authorB}' not found in the graph.")
    
    # Modified Ford-Fulkerson algorithm 
    modified_graph = subgraph.copy()
    min_cut = ford_fulkerson(modified_graph, authorA, authorB)
    
    # Actually removing the edges
    modified_graph.remove_edges_from(min_cut)
    
    return len(min_cut)


### Functionality 5 - Extracting Communities

Input: 
- The graph data 
- $N$: denoting the top $N$ papers that their data should be considered
- Paper\_1: denoting the name of one of the papers 
- Paper\_2: denoting the name of one of the papers

Output:
- The minimum number of edges that should be removed to form communities
- A list of communities, each containing a list of papers that belong to them.
- Whether the Paper\_1 and Paper\_2 belongs to the same community. 

------------------------------------------------------------------------

this function systematically examines every edge in the graph, determining whether it connects nodes from distinct communities. By doing so, it calculates the count of edges that would need to be cut to achieve the desired partition of the graph into communities.

In [92]:
def count_cut_edges(graph, communities):
    cut_edges = 0
    
    for edge in graph.edges():
        node1, node2 = edge
        community_node1 = get_community(node1, communities)
        community_node2 = get_community(node2, communities)
        # check if they are in the same community
        if community_node1 != community_node2:
            cut_edges += 1
    
    return cut_edges

def get_community(node, communities):
    # Find the community to which the node belongs
    for community, nodes in enumerate(communities):
        if node in nodes:
            return community
    
    return None  # Node not found in any community

This functionality uses Clauset-Newman-Moore greedy modularity maximization to find the community partition with the largest modularity.

The modularity of a graph given a partition (the communities) is a parameter that quantifies the degree of internal connectivity within communities compared to connections between communities. the formula is:
$$
Mod = \sum_{c=1}^{n} \left( \frac{L_c}{m} - \gamma \left( \frac{k_c}{2m} \right)^2 \right)
$$
where the sum iterates over all communities $c$, $m$ is the number of edges, $L_c$ is the number of intra-community links for community $c$, $k_c$ is the sum of degrees of the nodes in community $c$, and $\gamma$ is the resolution parameter (in this case 1, but it could be changed).

Greedy modularity maximization begins with each node in its own community and repeatedly joins the pair of communities that lead to the largest modularity until no further increase in modularity is possible 


In [93]:

def greedy_modularity_communities(G):
    # First create one community for each node
    communities = [frozenset([u]) for u in G.nodes()]
    # Track merges
    merges = []
    
    old_modularity = None
    new_modularity = nx.algorithms.community.modularity(G, communities)
    
    # Merge communities until no improvement is possible
    while old_modularity is None or new_modularity > old_modularity:
        # Save modularity for comparison
        old_modularity = new_modularity
        # Find best pair to merge
        trial_communities = list(communities)
        to_merge = None
        for i, u in enumerate(communities):
            for j, v in enumerate(communities):
                # Skip i==j and empty communities
                if j <= i or len(u) == 0 or len(v) == 0:
                    continue
                # Merge communities u and v
                trial_communities[j] = u | v
                trial_communities[i] = frozenset([])
                trial_modularity = nx.algorithms.community.modularity(G, trial_communities)
                if trial_modularity >= new_modularity:
                    # Check if strictly better or tie
                    if trial_modularity > new_modularity:
                        # Found new best, save modularity and group indexes
                        new_modularity = trial_modularity
                        to_merge = (i, j, new_modularity - old_modularity)
                    elif to_merge and min(i, j) < min(to_merge[0], to_merge[1]):
                        # Break ties by choosing pair with lowest min id
                        new_modularity = trial_modularity
                        to_merge = (i, j, new_modularity - old_modularity)
                # Un-merge
                trial_communities[i] = u
                trial_communities[j] = v
        if to_merge is not None:
            # If the best merge improves modularity, use it
            merges.append(to_merge)
            i, j, dq = to_merge
            u, v = communities[i], communities[j]
            communities[j] = u | v
            communities[i] = frozenset([])
    # Remove empty communities and sort
    return sorted((c for c in communities if len(c) > 0), key=len, reverse=True)


This selection was deliberate, distinguishing it from alternative methods such as Newman's algorithm, which ceases when the graph is partitioned into disconnected subgraphs. Utilizing Newman's algorithm on our graph, for instance, would terminate prematurely due to the existence of multiple disconnected subgraphs, resulting in a partition that merely replicates these subgraphs as communities. The Clauset-Newman-Moore approach, on the other hand, ensures a partition with favorable characteristics, avoiding premature halting and providing a more meaningful community structure even in the presence of disconnected subgraphs.

In [94]:
def functionality_5(graph, paper_1, paper_2, N):
    
    # Extract the top N authors according to degree and create a subgraph
    top_papers = sorted(graph.degree, key=lambda x: x[1], reverse=True)[:N]
    top_nodes = [author[0] for author in top_papers]
    subgraph = graph.subgraph(top_nodes)
    
    # Check if paper_1 and paper_2 are nodes of the graph (Top N)
    if paper_1 not in subgraph.nodes:
        raise ValueError(f"Node '{paper_1}' not found in the graph.")

    if paper_2 not in subgraph.nodes:
        raise ValueError(f"Node '{paper_2}' not found in the graph.")
    
    # Using the Clauset-Newman-Moore greedy modularity maximization
    c = list(greedy_modularity_communities(subgraph.copy()))
    
    # find the nodes forming the communities
    communities = []
    for i in c:
        communities.append(list(i))

    # Check if Paper_1 and Paper_2 belong to the same community
    same_community = any(paper_1 in community and paper_2 in community for community in communities)
    
    # Using the above function to get the number of edges cut
    number_edges_cut = count_cut_edges(subgraph, communities)
    
    return number_edges_cut, communities, same_community
